## Raghavendra Deshmukh - 8854506¶

**Assignment 3 CSCN8000 Artificial Intelligence Algorithms and Mathematics**

1.  Use iris flower dataset from sklearn library and try to form clusters of flowers using petal width and length features. Drop the other two features for simplicity.
   
    Figure out if any preprocessing such as scaling would help here
    
    Draw elbow plot and from that figure out optimal value of k
    
2.  Use the heart dataset from the Resources Folder or access it from https://www.kaggle.com/fedesoriano/heart-failure-prediction 
   
    Load heart disease dataset in pandas dataframe
   
    Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
    
    Convert text columns to numbers using label encoding / one hot encoding
    
    Apply scaling
    
    Build a classification model using various methods (SVM, logistic regression, random forest) and check which model gives you the best accuracy
    
    Now use PCA to reduce dimensions, retrain your model and see its impact on your model in terms of accuracy. 
  



**Notes: This lab should be submitted as a notebook and an HTML. Follow https://docs.github.com/en/pages/quickstart.**

### Part - 1
#### Use iris flower dataset from sklearn library and try to form clusters of flowers using petal width and length features. Drop the other two features for simplicity.

#### Import Necessary Libraries

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load the Iris flower dataset
iris = load_iris()
data = iris.data
features = iris.feature_names

# Create a DataFrame with petal length and width features
df = pd.DataFrame(data, columns=features)
petal_data = df[['petal length (cm)', 'petal width (cm)']]

# Preprocess the data - Scaling
scaler = StandardScaler()
scaled_data = scaler.fit_transform(petal_data)

# Find optimal number of clusters using elbow method
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(range(1, 11))
plt.show()


<Figure size 800x500 with 1 Axes>

#### Based on the elbow plot, perform clustering on k

In [49]:
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(scaled_data)
df['cluster'] = clusters
df

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),cluster
0,5.1,3.5,1.4,0.2,2
1,4.9,3.0,1.4,0.2,2
2,4.7,3.2,1.3,0.2,2
3,4.6,3.1,1.5,0.2,2
4,5.0,3.6,1.4,0.2,2
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


### Part - 2
#### Use the heart dataset from the Resources Folder or access it from https://www.kaggle.com/fedesoriano/heart-failure-prediction

#### Load the Heart Disease Dataset

In [20]:
# Load dataset
heart_data = pd.read_csv('heart_failure_dataset.csv')
# Display the first few rows of the dataset
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


####  Removing Outliers using Z-Score

In [21]:
from scipy.stats import zscore

# Calculate Z-scores for each feature
z_scores = zscore(heart_data.select_dtypes(include=['float64', 'int64']))

# Define a threshold for outliers (usually Z-score > 3 or < -3)
threshold = 3
outliers = (z_scores > threshold) | (z_scores < -threshold)

# Remove rows with outliers
heart_data = heart_data[~outliers.any(axis=1)]

# Display the shape of the cleaned dataset
print("Shape after removing outliers:", heart_data.shape)


Shape after removing outliers: (899, 12)


#### Convert Text Columns to Numbers using Label Encoding

In [22]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Convert categorical columns to numerical values using Label Encoding
categorical_columns = ['ChestPainType', 'RestingECG', 'ST_Slope']
for column in categorical_columns:
    heart_data[column] = label_encoder.fit_transform(heart_data[column])

# Display the first few rows of the dataset after label encoding
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,1,140,289,0,1,172,N,0.0,2,0
1,49,F,2,160,180,0,1,156,N,1.0,1,1
2,37,M,1,130,283,0,2,98,N,0.0,2,0
3,48,F,0,138,214,0,1,108,Y,1.5,1,1
4,54,M,2,150,195,0,1,122,N,0.0,2,0


#### Apply Scaling

In [23]:
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,1,140,289,0,1,172,N,0.0,2,0
1,49,F,2,160,180,0,1,156,N,1.0,1,1
2,37,M,1,130,283,0,2,98,N,0.0,2,0
3,48,F,0,138,214,0,1,108,Y,1.5,1,1
4,54,M,2,150,195,0,1,122,N,0.0,2,0


In [24]:

heart_data['ExerciseAngina'] = heart_data['ExerciseAngina'].replace({'N': 0, 'Y': 1})

In [25]:
heart_data['Sex'] = heart_data['Sex'].replace({'M': 0, 'F': 1})

0      0
1      1
2      0
3      1
4      0
      ..
913    1
914    1
915    1
916    1
917    0
Name: HeartDisease, Length: 899, dtype: int64

In [31]:
from sklearn.preprocessing import StandardScaler

# Split data into features (X) and target (y)
X = heart_data.drop(columns=['HeartDisease'])
y = heart_data['HeartDisease']

# Initialize StandardScaler
scaler = StandardScaler()

# Scale the features (excluding the target column)

heart_data_scaled = scaler.fit_transform(X)
heart_data_scaled = pd.DataFrame(heart_data_scaled, columns=X.columns)

# Display the scaled dataset
heart_data_scaled.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,-1.428154,-0.515943,0.224572,0.465900,0.849636,-0.550362,0.014143,1.384320,-0.822945,-0.855469,1.042496
1,-0.475855,1.938199,1.270637,1.634714,-0.168122,-0.550362,0.014143,0.752973,-0.822945,0.137516,-0.622165
2,-1.745588,-0.515943,0.224572,-0.118507,0.793612,-0.550362,1.603415,-1.535661,-0.822945,-0.855469,1.042496
3,-0.581666,1.938199,-0.821492,0.349019,0.149344,-0.550362,0.014143,-1.141069,1.215148,0.634008,-0.622165
4,0.053200,-0.515943,1.270637,1.050307,-0.028064,-0.550362,0.014143,-0.588640,-0.822945,-0.855469,1.042496


#### Build Classification Models

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into features (X) and target (y)
X = heart_data_scaled.iloc[:, :-1]
#y = heart_data_scaled.iloc[:, -1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialize and train different classifiers
classifiers = {
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier()
}

# Train and evaluate each model
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

# Display the model accuracies
for name, accuracy in results.items():
    print(f"{name}: Accuracy = {accuracy:.2f}")


SVM: Accuracy = 0.79
Logistic Regression: Accuracy = 0.81
Random Forest: Accuracy = 0.79


#### Apply PCA for Dimensionality Reduction

In [37]:
from sklearn.decomposition import PCA

# Initialize PCA
pca = PCA(n_components=0.95)  # Retain 95% variance

# Apply PCA on the scaled features
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Initialize and train classifiers on PCA-transformed data
results_pca = {}
for name, clf in classifiers.items():
    clf.fit(X_train_pca, y_train)
    y_pred_pca = clf.predict(X_test_pca)
    accuracy_pca = accuracy_score(y_test, y_pred_pca)
    results_pca[name] = accuracy_pca

# Display the model accuracies after PCA
for name, accuracy_pca in results_pca.items():
    print(f"{name} (PCA): Accuracy = {accuracy_pca:.2f}")


SVM (PCA): Accuracy = 0.82
Logistic Regression (PCA): Accuracy = 0.81
Random Forest (PCA): Accuracy = 0.77


#### CONCLUSION

* The application of PCA yielded diverse outcomes across distinct models. While Logistic Regression experienced modest improvements through dimensionality reduction, the performance of Random Forest was adversely affected. SVM, however, demonstrated consistent performance in both situations.

* It is crucial to emphasize that PCA primarily aims at reducing dimensions, which can contribute to expedited training durations and diminished overfitting. The influence on accuracy can vary according to the individual characteristics and connections present within the data.